In [1]:
!fuser -k 8000/tcp
!curl -fsSL https://ollama.com/install.sh | sh
!pip install fastapi uvicorn langchain langchain-ollama langchain-core python-telegram-bot requests pandas nest_asyncio pyngrok pyTelegramBotAPI openai-whisper ffmpeg-python python-dotenv langchain-community
print("--- Instalações concluídas ---")

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
--- Instalações concluídas ---


In [2]:
NGROK_TOKEN = "31vi0WGJCCMtEafYXVqn81SlJaV_sMhz6UBjNzUkGsRvHurg"

env_content = f"""
TELEGRAM_TOKEN=8574612472:AAGd98wI3oVpeqY4ZYobI43hXoNpcm2_mYQ
WEATHERSTACK_KEY=b5242e8d7fc81eb3d03021715fe58723
GEOAPIFY_KEY=bd5b5268c7124fd6bde99d3a4c0a4176
EXCHANGERATE_KEY=8e0ac77b0bc2293dbd8d60e3
NGROK_TOKEN={NGROK_TOKEN}
"""
with open(".env", "w") as f:
    f.write(env_content)
!ngrok config add-authtoken {NGROK_TOKEN}

print("--- Arquivo .env criado e Ngrok configurado ---")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
--- Arquivo .env criado e Ngrok configurado ---


In [3]:
import sqlite3
import pandas as pd
import os
import requests
import threading
import nest_asyncio
import uvicorn
import time
import subprocess
from fastapi import FastAPI, Request
from pyngrok import ngrok
from dotenv import load_dotenv
import telebot
import whisper
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import tool

DB_PATH = "tourism.db"

destinos = [
    ("Paris", "França", "Patisserie, Queijos", "Temperado", "Torre Eiffel, Museu do Louvre"),
    ("Tóquio", "Japão", "Sushi, Ramen", "Temperado", "Templo Senso-ji, Shibuya Crossing"),
    ("Rio de Janeiro", "Brasil", "Feijoada, Caipirinha", "Tropical", "Cristo Redentor, Pão de Açúcar"),
    ("Roma", "Itália", "Pasta, Gelato", "Mediterrâneo", "Coliseu, Vaticano"),
]

def setup_database():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("""
    CREATE TABLE IF NOT EXISTS destinos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        pais TEXT NOT NULL,
        culinaria TEXT NOT NULL,
        clima TEXT NOT NULL,
        atracoes TEXT NOT NULL
    )
    """)
    c.execute("SELECT COUNT(*) FROM destinos")
    count = c.fetchone()[0]
    if count == 0:
        c.executemany(
            "INSERT INTO destinos (nome, pais, culinaria, clima, atracoes) VALUES (?, ?, ?, ?, ?)",
            destinos
        )
        conn.commit()
        print(f"{len(destinos)} destinos inseridos.")
    else:
        print("Banco de dados de destinos já possui dados.")
    conn.close()

setup_database()

print("--- Banco de Dados pronto ---")

Banco de dados de destinos já possui dados.
--- Banco de Dados pronto ---


In [4]:
load_dotenv()

app = FastAPI(title="Guia Turístico API")
BASE = "http://localhost:8000"

WEATHERSTACK_KEY = os.getenv("WEATHERSTACK_KEY")
GEOAPIFY_KEY = os.getenv("GEOAPIFY_KEY")
EXCHANGERATE_KEY = os.getenv("EXCHANGERATE_KEY")
CURRENCY_API_URL = f"https://v6.exchangerate-api.com/v6/{EXCHANGERATE_KEY}/latest"

def get_connection():
    return sqlite3.connect(DB_PATH)

def query_db(query, params=()):
    conn = get_connection()
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute(query, params)
    rows = cur.fetchall()
    conn.close()
    return [dict(row) for row in rows]

@app.get("/api/destinos")
def list_destinos(limit: int = 50):
    rows = query_db("SELECT nome, pais, culinaria, clima, atracoes FROM destinos LIMIT ?", (limit,))
    return rows

@app.get("/api/clima/{cidade}")
def clima_atual(cidade: str):
    if not WEATHERSTACK_KEY:
        return {"erro": "Chave Weatherstack não configurada."}
    url = f"http://api.weatherstack.com/current?access_key={WEATHERSTACK_KEY}&query={cidade}&lang=pt"
    resp = requests.get(url)
    data = resp.json()
    if "current" not in data:
        return {"erro": data.get("error", {}).get("info", "Cidade inválida ou limite atingido.")}
    return {
        "cidade": data["location"]["name"], "pais": data["location"]["country"],
        "descricao": data["current"]["weather_descriptions"][0],
        "temperatura_c": data["current"]["temperature"],
        "sensacao_c": data["current"]["feelslike"],
    }

@app.get("/api/conversao_moeda/{base}/{destino}")
def conversao_moeda(base: str, destino: str):
    if not EXCHANGERATE_KEY:
        return {"erro": "Chave Exchangerate não configurada."}
    url = f"{CURRENCY_API_URL}/{base.upper()}"
    resp = requests.get(url)
    data = resp.json()
    if data.get("result") == "error":
         return {"erro": f"API de Câmbio: {data.get('error-type')}"}
    taxa = data.get("conversion_rates", {}).get(destino.upper())
    if not taxa:
        return {"erro": f"Moeda destino '{destino.upper()}' inválida."}
    return {"base": base.upper(), "destino": destino.upper(), "taxa": taxa}

@app.get("/api/culinaria/{cidade}")
def culinaria_local(cidade: str):
    if not GEOAPIFY_KEY:
        return {"erro": "Chave Geoapify não configurada."}
    geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={cidade}&limit=1&apiKey={GEOAPIFY_KEY}"
    geo_resp = requests.get(geocode_url)
    if geo_resp.status_code != 200 or not geo_resp.json().get("features"):
        return {"erro": "Cidade não encontrada para buscar culinária."}
    coords = geo_resp.json()["features"][0]["geometry"]["coordinates"]
    lon, lat = coords[0], coords[1]
    url = f"https://api.geoapify.com/v2/places?categories=catering.restaurant&filter=circle:{lon},{lat},5000&bias=proximity:{lon},{lat}&limit=5&apiKey={GEOAPIFY_KEY}"
    resp = requests.get(url)
    data = resp.json()
    lugares = [
        f["properties"].get("name", "Restaurante sem nome")
        for f in data.get("features", []) if f.get("properties", {}).get("name")
    ]
    return {"cidade": cidade, "restaurantes_sugeridos": lugares}

@app.get("/api/atracoes/{cidade}")
def atracoes_turisticas(cidade: str):
    if not GEOAPIFY_KEY:
        return {"erro": "Chave Geoapify não configurada."}
    geocode_url = f"https://api.geoapify.com/v1/geocode/search?text={cidade}&limit=1&apiKey={GEOAPIFY_KEY}"
    geo_resp = requests.get(geocode_url)
    if geo_resp.status_code != 200 or not geo_resp.json().get("features"):
        return {"erro": "Cidade não encontrada para buscar atrações."}
    coords = geo_resp.json()["features"][0]["geometry"]["coordinates"]
    lon, lat = coords[0], coords[1]
    url = f"https://api.geoapify.com/v2/places?categories=tourism.attraction&filter=circle:{lon},{lat},5000&bias=proximity:{lon},{lat}&limit=5&apiKey={GEOAPIFY_KEY}"
    resp = requests.get(url)
    data = resp.json()
    atracoes = [
        f["properties"].get("name", "Atração sem nome")
        for f in data.get("features", []) if f.get("properties", {}).get("name")
    ]
    return {"cidade": cidade, "atracoes_sugeridas": atracoes}

print("--- Definições da API FastAPI prontas ---")

--- Definições da API FastAPI prontas ---


In [5]:
!fuser -k 11434/tcp
!nohup ollama serve > /dev/null 2>&1 &
!sleep 5
!ollama pull qwen2:7b
!ollama list



NAME        ID              SIZE      MODIFIED               
qwen2:7b    dd314f039b9d    4.4 GB    Less than a second ago    


In [6]:
from langchain_ollama import ChatOllama

MODEL_NAME = os.getenv("OLLAMA_MODEL", "qwen2:7b")
try:
    model = ChatOllama(model=MODEL_NAME)
    model.invoke("teste")
    print(f"✅ Modelo Ollama '{MODEL_NAME}' conectado e funcional.")
except Exception as e:
    print(f"❌ ERRO: Não foi possível conectar ao modelo Ollama '{MODEL_NAME}'.")
    print("Verifique se o Ollama está rodando ou se o modelo foi baixado.")
    print("Erro técnico:", e)
    from langchain.llms.fake import FakeListLLM
    model = FakeListLLM(responses=["Desculpe, o modelo de IA não está disponível agora."])
    print("⚠️ Usando modelo falso (FakeListLLM) como fallback.")


@tool
def buscar_destino_catalogo(cidade: str) -> dict:
    """Busca informações gerais de um destino (clima, culinária, atrações) no catálogo local."""
    print(f"--- Agente chamou: buscar_destino_catalogo(cidade='{cidade}') ---")
    destinos = query_db("SELECT * FROM destinos WHERE LOWER(nome) = LOWER(?)", (cidade,))
    if not destinos:
        return {"erro": f"Não encontrei a cidade {cidade} no catálogo."}
    return destinos[0]


@tool
def get_clima_tempo_real(cidade: str) -> dict:
    """Obtém o clima atual em tempo real de uma cidade."""
    print(f"--- Agente chamou: get_clima_tempo_real(cidade='{cidade}') ---")
    r = requests.get(f"{BASE}/api/clima/{cidade}")
    return r.json()


@tool
def get_culinaria_local(cidade: str) -> dict:
    """Retorna até 5 restaurantes e tipos de culinária local de uma cidade."""
    print(f"--- Agente chamou: get_culinaria_local(cidade='{cidade}') ---")
    r = requests.get(f"{BASE}/api/culinaria/{cidade}")
    return r.json()


@tool
def get_atracoes_turisticas(cidade: str) -> dict:
    """Lista até 5 atrações turísticas populares de uma cidade."""
    print(f"--- Agente chamou: get_atracoes_turisticas(cidade='{cidade}') ---")
    r = requests.get(f"{BASE}/api/atracoes/{cidade}")
    return r.json()


@tool
def get_conversao_moeda(valor: float, moeda_base: str, moeda_destino: str) -> dict:
    """Converte um valor monetário entre duas moedas (ex: USD → BRL)."""
    print(f"--- Agente chamou: get_conversao_moeda(valor={valor}, moeda_base='{moeda_base}', moeda_destino='{moeda_destino}') ---")
    r = requests.get(f"{BASE}/api/conversao_moeda/{moeda_base}/{moeda_destino}")
    data = r.json()
    if "taxa" in data:
        resultado = valor * data["taxa"]
        return {
            "valor_base": valor,
            "moeda_base": moeda_base.upper(),
            "valor_destino": round(resultado, 2),
            "moeda_destino": moeda_destino.upper()
        }
    return data


tools = [
    buscar_destino_catalogo,
    get_clima_tempo_real,
    get_culinaria_local,
    get_atracoes_turisticas,
    get_conversao_moeda
]


template = """
Você é um guia de viagens e turismo que fala apenas português.
Seu estilo de resposta é organizado, simpático e informativo.

Sempre que listar restaurantes, siga este formato:
  Nome do restaurante
  Tipo de culinária (ex: francesa, italiana, frutos do mar, vegana, etc.)
  Breve descrição do que o restaurante costuma servir ou do ambiente.

Se o tipo de culinária não for fornecido pela ferramenta, use seu conhecimento geral sobre o país ou região
para inferir o tipo mais provável.

Exemplo de formato ideal:
---
 Restaurantes recomendados em Paris:
1. Crêperie du Cloître — Culinária francesa, especializada em crepes doces e salgados.
2. Le Petit Châtelet — Cozinha francesa clássica, conhecida por carnes grelhadas e vinhos locais.
3. La Bûcherie — Pratos vegetarianos e ambiente alternativo próximo à Catedral de Notre-Dame.
---

Sempre apresente as respostas em tópicos organizados e, destaque os tópicos.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{pergunta}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

model_with_tools = model.bind_tools(tools)
agent = create_tool_calling_agent(llm=model_with_tools, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("🤖 --- Agente LangChain (Cérebro) pronto ---")


✅ Modelo Ollama 'qwen2:7b' conectado e funcional.
🤖 --- Agente LangChain (Cérebro) pronto ---


In [ ]:
TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")
if not TELEGRAM_TOKEN:
    print("ERRO FATAL: TELEGRAM_TOKEN não encontrado no .env")
else:
    bot = telebot.TeleBot(TELEGRAM_TOKEN)
    print("Bot do Telegram inicializado.")

    def whisper_transcribe(file_path: str, model_size="small") -> str:
        try:
            wmodel = whisper.load_model(model_size)
            result = wmodel.transcribe(file_path)
            return result["text"]
        except Exception as e:
            print(f"Erro no Whisper: {e}")
            return "[Erro na transcrição]"

    def get_agent_response(pergunta: str) -> str:
        """Função central que chama o agente."""
        try:
            resposta = agent_executor.invoke({"pergunta": pergunta})
            return resposta["output"]
        except Exception as e:
            print(f"\nERRO NO AGENTE: {e}")
            return "Desculpe, tive um problema ao processar sua solicitação com o modelo de IA. Tente novamente."

    @bot.message_handler(commands=['start', 'help'])
    def send_welcome(message):
        bot.reply_to(message, "Olá! Eu sou seu Guia Turístico. Pergunte-me sobre destinos, clima, culinária ou conversão de moedas.")

    @bot.message_handler(func=lambda message: True, content_types=['text'])
    def handle_text(message):
        pergunta = message.text
        print(f"[Telegram] Pergunta (texto): {pergunta}")

        bot.send_chat_action(message.chat.id, 'typing')
        resposta = get_agent_response(pergunta)

        bot.send_message(message.chat.id, resposta, parse_mode="Markdown")

    @bot.message_handler(content_types=['voice'])
    def handle_voice(message):
        bot.reply_to(message, "Recebi seu áudio. Transcrevendo...")
        file_id = message.voice.file_id
        file_info = bot.get_file(file_id)
        file_url = f"https://api.telegram.org/file/bot{TELEGRAM_TOKEN}/{file_info.file_path}"

        local_path = f"{file_id}.ogg"
        audio_data = requests.get(file_url).content
        with open(local_path, "wb") as f:
            f.write(audio_data)

        texto = whisper_transcribe(local_path)
        bot.reply_to(message, f"Você disse: \"{texto}\"\nPensando...")

        resposta = get_agent_response(texto)
        bot.reply_to(message, resposta)

    @app.post("/telegram_webhook")
    async def telegram_webhook(req: Request):
        data = await req.json()
        bot.process_new_updates([telebot.types.Update.de_json(data)])
        return {"ok": True}

    HOST = "0.0.0.0"
    PORT = 8000

    nest_asyncio.apply()

    def run_ollama_serve():
        subprocess.Popen(["ollama", "serve"])

    print("Iniciando Ollama em background...")
    thread_ollama = threading.Thread(target=run_ollama_serve)
    thread_ollama.start()
    time.sleep(5)

    def run_server():
        uvicorn.run(app, host=HOST, port=PORT)

    print("Iniciando servidor FastAPI em background...")
    thread_api = threading.Thread(target=run_server)
    thread_api.start()
    time.sleep(3)

    print("Iniciando Ngrok...")
    public_url = ngrok.connect(PORT)
    print(f"API pública (Ngrok) disponível em: {public_url.public_url}")

    bot.remove_webhook()
    WEBHOOK_URL = f"{public_url.public_url}/telegram_webhook"
    bot.set_webhook(url=WEBHOOK_URL)
    print("Webhook do Telegram configurado com sucesso!")
    print("--- SEU BOT ESTÁ ONLINE NO TELEGRAM ---")
    print("--- Deixe esta célula rodando ---")

    try:
        while True:
            time.sleep(60)
    except KeyboardInterrupt:
        print("Desligando servidores...")
        ngrok.disconnect(public_url.public_url)
        print("Ngrok desconectado.")

Bot do Telegram inicializado.
Iniciando Ollama em background...


INFO:     Started server process [22830]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Iniciando servidor FastAPI em background...
Iniciando Ngrok...
API pública (Ngrok) disponível em: https://1e4874b05539.ngrok-free.app
Webhook do Telegram configurado com sucesso!
--- SEU BOT ESTÁ ONLINE NO TELEGRAM ---
--- Deixe esta célula rodando ---
INFO:     91.108.5.76:0 - "POST /telegram_webhook HTTP/1.1" 200 OK
INFO:     91.108.5.76:0 - "POST /telegram_webhook HTTP/1.1" 200 OK
[Telegram] Pergunta (texto): Olá, qual o clima de Paris agora


> Entering new AgentExecutor chain...

Invoking: `get_clima_tempo_real` with `{'cidade': 'Paris'}`


--- Agente chamou: get_clima_tempo_real(cidade='Paris') ---
INFO:     127.0.0.1:47234 - "GET /api/clima/Paris HTTP/1.1" 200 OK
{'cidade': 'Paris', 'pais': 'France', 'descricao': 'Sunny', 'temperatura_c': 17, 'sensacao_c': 17}O clima atual em Paris é ensolarado. A temperatura está em 17 graus Celsius e a sensação térmica também é de 17 graus.

> Finished chain.
